#### pip install spacy openpyxl
#### python -m spacy download en_core_web_lg


In [6]:
import nltk
from nltk.corpus import wordnet as wn

nltk.download("wordnet")


def add_synonyms(text):
    expanded_text = []
    for word in text.split():
        synonyms = {
            lemma.name() for synset in wn.synsets(word) for lemma in synset.lemmas()
        }
        expanded_text.append(" ".join(synonyms))
    return " ".join(expanded_text)


# Assuming nlp has been loaded and preprocess_text is defined
def preprocess_and_expand_text(text):
    lemmatized_text = preprocess_text(text)
    expanded_text = add_synonyms(lemmatized_text)
    return expanded_text

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...


In [7]:
categories_df["processed_description"] = categories_df["description"].apply(
    preprocess_and_expand_text
)
domains_df["processed_description"] = domains_df["description"].apply(
    preprocess_and_expand_text
)

NameError: name 'preprocess_text' is not defined

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

# Combine the descriptions to train the vectorizer
all_descriptions = pd.concat(
    [categories_df["processed_description"], domains_df["processed_description"]]
)
vectorizer.fit(all_descriptions)

# Transform descriptions into TF-IDF vectors
categories_df["tfidf_vector"] = list(
    vectorizer.transform(categories_df["processed_description"]).toarray()
)
domains_df["tfidf_vector"] = list(
    vectorizer.transform(domains_df["processed_description"]).toarray()
)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


def find_best_match(cat_tfidf_vec, domain_tfidf_vectors):
    similarities = cosine_similarity([cat_tfidf_vec], domain_tfidf_vectors)[0]
    return similarities.argmax()


# Convert domain vectors to a matrix for efficiency
domain_tfidf_matrix = np.array(domains_df["tfidf_vector"].tolist())

# Find the best match for each category
categories_df["best_match_domain_index"] = categories_df["tfidf_vector"].apply(
    lambda x: find_best_match(x, domain_tfidf_matrix)
)

# Map the index to the actual domain name
categories_df["best_match_domain"] = categories_df["best_match_domain_index"].apply(
    lambda x: domains_df.iloc[x]["domain"]
)

,label,description,lemmatized_description,description_vector,best_match_label_index,best_match_label
0,ATVs Dealers & Services,Businesses categorized under ATVs Dealers & Se...,business categorize atv Dealers Services speci...,"(business, categorize, atv, Dealers, Services,...",36,Motor Vehicle and Parts Dealers
1,Abortion Clinics,Abortion Clinics provide medical services rela...,Abortion Clinics provide medical service relat...,"(Abortion, Clinics, provide, medical, service,...",76,Hospitals
2,Accounting & Bookkeeping Services,Accounting & Bookkeeping Services encompass a ...,Accounting Bookkeeping Services encompass rang...,"(Accounting, Bookkeeping, Services, encompass,...",72,Administrative and Support Services
3,Acupuncture clinic,An Acupuncture clinic is a healthcare facility...,acupuncture clinic healthcare facility special...,"(acupuncture, clinic, healthcare, facility, sp...",76,Hospitals
4,Adhesives & Sealants,Adhesives & Sealants encompass a diverse range...,adhesive Sealants encompass diverse range prod...,"(adhesive, Sealants, encompass, diverse, range...",32,Miscellaneous Manufacturing
5,Adoption Agencies & Foster Care,Adoption Agencies & Foster Care involve organi...,Adoption Agencies Foster Care involve organiza...,"(Adoption, Agencies, Foster, Care, involve, or...",77,Nursing and Residential Care Facilities
6,Adult & Porn,The Adult & Porn category encompasses business...,Adult Porn category encompass business service...,"(Adult, Porn, category, encompass, business, s...",56,Motion Picture and Sound Recording Industries
7,Adult Education Schools,Adult Education Schools provide specialized ed...,Adult Education Schools provide specialized ed...,"(Adult, Education, Schools, provide, specializ...",74,Educational Services
8,Adult Entertainment Clubs,Adult Entertainment Clubs typically refer to e...,Adult Entertainment Clubs typically refer esta...,"(Adult, Entertainment, Clubs, typically, refer...",79,"Performing Arts, Spectator Sports, and Related..."
9,Adult Escort Services,Adult Escort Services involve providing compan...,Adult Escort Services involve provide companio...,"(Adult, Escort, Services, involve, provide, co...",78,Social Assistance


In [ ]:
categories_df